# Summarization of the completed procedure's final acts for the year 2025

## 1. LLM package install and import

In [ ]:
!pip install --no-deps "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.29" "trl<0.9.0" peft accelerate bitsandbytes
!pip install --upgrade --no-deps "transformers>=4.47.0" "tokenizers>=0.21.0" "huggingface-hub>=0.26.0,<1.0"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-1sler7sg/unsloth_76d26f278b1c459a812e89157e475082
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-1sler7sg/unsloth_76d26f278b1c459a812e89157e475082
  Resolved https://github.com/unslothai/unsloth.git to commit 8ea5338154859ed25b50366cb1264ed4d933eae3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.12.9-py3-none-any.whl size=382658 sha256=d1c0ab583b5dae2297921670865f280b49edf92c13879d3d226567500e80c2d2
  Stored in directory: /tmp/pip-ephem-wheel-cache-eda6cr7x/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 94.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 17.9 MB/s eta 0:00:

In [ ]:
import os
# To avoid 'MessageFactory' error on Kaggle
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
# Gives Torch priority to the GPU in comparison to Tenserflow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, BitsAndBytesConfig
import torch
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import concurrent.futures
import ast
import json
import re

# Make sure NLTK data is downloaded
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 2. Model loading

In [ ]:
model_name = "unsloth/phi-4-unsloth-bnb-4bit"

# Quantification configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model with the quantification configuration
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config, 
    device_map="auto",
    trust_remote_code=True
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.39G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/170 [00:00<?, ?B/s]

## 3. Data import

In [5]:
# To make it work here, first input the sample_cod_completed_2025.csv on Kaggle and name it completed-procedures-2025-27-12
df = pd.read_csv("/kaggle/input/completed-procedures-2025-27-12/sample_cod_completed_2025.csv") 

In [6]:
df

,Unnamed: 0.1,Unnamed: 0,url,reference,title,subjects,key_players,key_events,documentation_gateway,transparency,...,Mandatory consultation of other institutions,Other legal basis,legislative_proposal_url,legislative_proposal_text,decisions_url_list,resolution_text_1,position_text_1,text_adopted_others_1,final_act_text,final_act_url
0,12,12,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0023(COD),Insolvency proceedings: replacing Annexes A and B,"3.45.01 Company law, 3.45.03 Financial managem...",{'European Parliament': [{'Committee responsib...,{'Legislative proposal published': {'Date': '1...,"[{'Institution': 'European Commission', 'Docum...","{'Rapporteurs, Shadow Rapporteurs and Committe...",...,NaN,Rules of Procedure EP 165,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 12.2.2025 COM(...",['https://www.europarl.europa.eu/doceo/documen...,European Parliament legislative resolution of ...,Position of the European Parliament adopted at...,NaN,REGULATION (EU) 2025/2073 OF THE EUROPEAN PARL...,https://eur-lex.europa.eu/legal-content/EN/TXT...
1,13,13,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0012(COD),Additional customs duties on imports of certai...,6.20.03 Bilateral economic and trade agreement...,{'European Parliament': [{'Committee responsib...,{'Legislative proposal published': {'Date': '0...,"[{'Institution': 'European Commission', 'Docum...",NaN,...,NaN,Rules of Procedure EP 165,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 3.2.2025 COM(2...",['https://www.europarl.europa.eu/doceo/documen...,European Parliament legislative resolution of ...,Position of the European Parliament adopted at...,NaN,NaN,NaN
2,16,16,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0077(COD),Capital Requirements Regulation (CRR): require...,"2.50.03 Securities and financial markets, stoc...",{'European Parliament': [{'Committee responsib...,{'Legislative proposal published': {'Date': '3...,"[{'Institution': 'European Commission', 'Docum...","{'Rapporteurs, Shadow Rapporteurs and Committe...",...,[{'text': 'European Economic and Social Commit...,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 31.3.2025 COM(...",['https://www.europarl.europa.eu/doceo/documen...,European Parliament legislative resolution of ...,Position of the European Parliament adopted at...,NaN,REGULATION (EU) 2025/1215 OF THE EUROPEAN PARL...,https://eur-lex.europa.eu/legal-content/EN/TXT...
3,31,31,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0021(COD),Modification of customs duties applicable to i...,6.20.03 Bilateral economic and trade agreement...,{'European Parliament': [{'Committee responsib...,{'Legislative proposal published': {'Date': '2...,"[{'Institution': 'European Commission', 'Docum...","{'Rapporteurs, Shadow Rapporteurs and Committe...",...,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 28.1.2025 COM(...",['https://www.europarl.europa.eu/doceo/documen...,European Parliament legislative resolution of ...,Position of the European Parliament adopted at...,NaN,NaN,NaN
4,40,40,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0084(COD),"Amending ERDF, Cohesion Fund and Just Transiti...","4.70.01 Structural funds, investment funds in ...",{'European Parliament': [{'Committee responsib...,{'Legislative proposal published': {'Date': '0...,"[{'Institution': 'European Commission', 'Docum...",{'Other Members': [{'Name': [{'text': 'KATAINE...,...,[{'text': 'European Economic and Social Commit...,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Strasbourg, 1.4.2025 COM...",['https://www.europarl.europa.eu/doceo/documen...,European Parliament legislative resolution of ...,Position of the European Parliament adopted at...,{'ANNEX TO THE LEGISLATIVE RESOLUTION': {'text...,REGULATION (EU) 2025/1914 OF THE EUROPEAN PARL...,https://eur-lex.europa.eu/legal-content/EN/TX

## 4. Functions to summarize the procedures

In [7]:
# Wrap llm generation into a function
def generation(prompt) :
  model.generation_config.pad_token_id = tokenizer.pad_token_id

  messages = [
      {"role": "system", "content": "You are a journalist"},
      {"role": "user", "content": prompt},
  ]
  input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
  outputs = model.generate(input_tensor.to(model.device), max_new_tokens = 300, temperature=0.1, do_sample=True)

  result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)

  return result  

In [ ]:
# Function to prompt phi-4 to summarize the first page of a legislative final act
def paste_global_summary_first_page(first_page, final_act_title):
    prompt = f"""
You are a journalist explaining European legislation to the general public.

Below is the first page of the EP's final adopted act on the legislative procedure titled "{final_act_title}":

---
{first_page}
---

Summarize it in one or a few coherent paragraphs.
Your summary should:
- Explain the adopted act's main points.
- Remain concise, neutral, and clear. Do not repeat yourself.
- Avoid quoting specific articles or amendments.  
- Clarify technical or institutional terms in simple language.
- Exclude legal boilerplate: Systematically remove standard legal clauses that do not provide specific content to this law.

Base your explanation only on the provided text. Do not take sides. Do not defend nor attack the adopted text.
Output: a single concise paragraph.
"""

    result = generation(prompt)
    return result


In [ ]:
# Function prompting phi-4 to iteratively summarize the next page of a legislative final act with its previous summary
def paste_global_summary(final_act_title, previous_summary, new_text, i):
    prompt = f"""
You are a journalist explaining European legislation to the general public.

Below is the summary written so far, covering the first {i} pages of the final adopted act of the EP on the legislative procedure "{final_act_title}":

---
{previous_summary}
---

Here is the next page of the final adopted act:

---
{new_text}
---

Update and rewrite the summary so it now covers everything up to this point ({i+1} pages), in one or a few coherent paragraphs.
Your summary should:
- Explain the act's main points.
- Remain concise, neutral, and clear. Do not repeat yourself.
- Avoid quoting specific articles or amendments.  
- Clarify technical or institutional terms in simple language. 
- Exclude legal boilerplate: Systematically remove standard legal clauses that do not provide specific content to this law.


Base your summary only on the provided text. Do not take sides. Do not defend nor attack the adopted text.
Output: a single short paragraph.
"""

    result = generation(prompt)
    return result

In [10]:
# Function to split the legislative texts into chunks of 1000 words
def text_into_segments(text, max_words=1000):
    sentences = sent_tokenize(text)  # Split the text into sentences
    segments = []
    current_segment = ""
    current_word_count = 0

    # Iterate through each sentence and group them into segments
    for sentence in sentences:
        sentence_word_count = len(word_tokenize(sentence))
        # If adding this sentence exceeds the word limit, start a new segment
        if current_word_count + sentence_word_count > max_words:
            if current_segment:
                segments.append(current_segment.strip())
            current_segment = sentence
            current_word_count = sentence_word_count
        else:
            # Otherwise, keep adding sentences to the current segment
            current_segment += " " + sentence
            current_word_count += sentence_word_count

    # Append the last segment if any text remains
    if current_segment:
        segments.append(current_segment.strip())

    return segments

In [11]:
def process_text(idx, df):
    try:
        print(f"Processing procedure {df['reference'][idx]}")
        # Retrieve the raw legislative text
        text = str(df['final_act_text'][idx]) if pd.notna(df['final_act_text'][idx]) else ""

        if text.strip() == "":
            print(f"Error : 'final_act_text' is empty for procedure {df['reference'][idx]}")
            return {"title": df['title'][idx], "error": "final_act_text' is empty"}

        # Split the legislative text into segments for processing
        texts = text_into_segments(text)
        
        # Initialize the summary text
        summary = ""

        # Loop through each text segment and build the global summary incrementally
        for i, text in enumerate(texts):
            if i == 0:
                # Generate the first page summary
                summary = paste_global_summary_first_page(text, df['title'][idx])
            else:
                # Continue the summary with additional segments
                updated_summary = paste_global_summary(df['title'][idx], summary, text, i)
                if not updated_summary:
                    print(f"Summary failed for segment {i} of {df['title'][idx]}")
                else:
                    summary = updated_summary
        
        # Save the generated summary back into the DataFrame
        df.at[idx, 'final_act_summary'] = summary
        # Save the updated DataFrame to CSV after each summary
        df.to_csv('cod_final_act_general_summary.csv')
        print("Summary done")

    except Exception as e:
        print(e)
        # Print any error that occurs during processing
        return {
            "title": df['title'][idx],
            "error": str(e)
        }

In [12]:
# Iterate over all of the dataset's rows and process each legislative text
for idx in df.index:
    result = process_text(idx, df)
    del result
    # Clear GPU memory after each iteration to prevent memory overflow
    torch.cuda.empty_cache()

print('Summarization finished')

Processing procedure 2025/0023(COD)
Summary done
Processing procedure 2025/0012(COD)
Error : 'final_act_text' is empty for procedure 2025/0012(COD)
Processing procedure 2025/0077(COD)
Summary done
Processing procedure 2025/0021(COD)
Error : 'final_act_text' is empty for procedure 2025/0021(COD)
Processing procedure 2025/0084(COD)
Summary done
Processing procedure 2025/0085(COD)
Summary done
Processing procedure 2025/0058(COD)
Summary done
Processing procedure 2025/0039(COD)
Summary done
Processing procedure 2025/0051(COD)
Summary done
Processing procedure 2025/0188(COD)
Summary done
Processing procedure 2025/0129(COD)
Summary done
Processing procedure 2025/0104(COD)
Summary done
Processing procedure 2025/0526(COD)
Summary done
Processing procedure 2025/0044(COD)
Summary done
Processing procedure 2025/0070(COD)
Summary done
Processing procedure 2025/0022(COD)
Summary done
Processing procedure 2025/0074(COD)
Summary done
Processing procedure 2025/0056(COD)
Error : 'final_act_text' is emp